In [25]:
from datetime import datetime


from pydantic import (
    Field, 
    ConfigDict, 
    BaseModel, 
    ValidationError,
    ValidationInfo,
    field_validator
)

import uuid

from typing import Annotated, List, Optional
from pydantic.functional_validators import BeforeValidator
from pprint import pprint as print
# from fastapi import FastAPI, Body, HTTPException, status
# from fastapi.responses import Response
# from pydantic.functional_validators import BeforeValidator

# from typing_extensions import Annotated

# from bson import ObjectId
# import motor.motor_asyncio
# from pymongo import ReturnDocument


In [4]:

class Meeting(BaseModel):
    when: datetime
    where: bytes
    why: str = 'No idea'


m = Meeting(
    when='2020-01-01T12:00',
    where='home'
)

print(m.model_dump(exclude_unset=True))
#> {'when': datetime.datetime(2020, 1, 1, 12, 0), 'where': b'home'}
print(m.model_dump(exclude={'where'}, mode='json'))
#> {'when': '2020-01-01T12:00:00', 'why': 'No idea'}
print(m.model_dump_json(exclude_defaults=True))
#> {"when":"2020-01-01T12:00:00","where":"home"}


{'when': datetime.datetime(2020, 1, 1, 12, 0), 'where': b'home'}
{'when': '2020-01-01T12:00:00', 'why': 'No idea'}
{"when":"2020-01-01T12:00:00","where":"home"}


In [9]:


class Address(BaseModel):
    street: str
    city: str
    zipcode: str


class Meeting(BaseModel):
    when: datetime
    where: Address
    why: str = 'No idea'

    def schema(self):
        return self.model_json_schema()
    

m = Meeting(
    when='2020-01-01T12:00',
    where={'street': 'home', 'city': 'city', 'zipcode': '12345'}
)

print(m.schema())

{'$defs': {'Address': {'properties': {'street': {'title': 'Street', 'type': 'string'}, 'city': {'title': 'City', 'type': 'string'}, 'zipcode': {'title': 'Zipcode', 'type': 'string'}}, 'required': ['street', 'city', 'zipcode'], 'title': 'Address', 'type': 'object'}}, 'properties': {'when': {'format': 'date-time', 'title': 'When', 'type': 'string'}, 'where': {'$ref': '#/$defs/Address'}, 'why': {'default': 'No idea', 'title': 'Why', 'type': 'string'}}, 'required': ['when', 'where'], 'title': 'Meeting', 'type': 'object'}


In [32]:
PyObjectId = Annotated[str, BeforeValidator(str)]


class Workflows(BaseModel):
    id: Optional[PyObjectId] = Field(alias="_id", default=None)
    name: str = Field()
    status: str = Field(default='pending')
    done: int = Field(default=0)         # should this be a bool?
    total: int = Field(default=1)           # what is this?
    started_at: datetime = Field(default=datetime.now())
    completed_at: datetime = Field(default=None)
    model_config = ConfigDict(
        populate_by_name=True,
        arbitrary_types_allowed=True,
        json_schema_extra={
            'example': {
                'name': 'workflow_name',
                'status': 'pending',
                'done': 0,
                'total': 1,
                'started_at': '2021'
            }
        }
    )

    @field_validator('done','total', mode = "before")
    @classmethod
    def field_must_be_positive(cls, v, info: ValidationInfo):
        field = info.field_name
        if v < 0:
            raise ValueError(f'`{field}` must be positive')
        return v

    
    
w = Workflows(
    name = str(uuid.uuid4()),
)

print(w.model_dump())


{'completed_at': None,
 'done': 0,
 'id': None,
 'name': 'fd998441-c8ad-4cb1-ac3b-2071c50a3c1f',
 'started_at': datetime.datetime(2024, 4, 7, 12, 32, 36, 737731),
 'status': 'pending',
 'total': 1}
